# Kütüphane Verilerinin Ön İşlenmesi

İBB kütüphane verilerinden Photon API ile koordinat çekip JSON formatında kaydetme.

## Adım 1: Kütüphaneleri İçe Aktarma

In [ ]:
import pandas as pd
import time
import requests

pd.set_option('display.max_colwidth', None)

## Adım 2: Veri Yükleme

In [67]:
# Excel dosyasını yükle
df = pd.read_excel(r"kütüphaneler(ibb).xlsx")
print(f"Toplam kütüphane sayısı: {len(df)}")

Toplam kütüphane sayısı: 60


## Adım 3: Geocoding Fonksiyonları

In [ ]:
def get_coordinates_from_photon(address):
    """
    Photon API kullanarak adres için enlem ve boylam döndürüyoruz
    """
    try:
        # Photon API'ye istek gönder
        url = "https://photon.komoot.io/api/"
        params = {'q': address, 'limit': 1}
        response = requests.get(url, params=params, timeout=10)
        
        # Yanıtı kontrol et
        if response.status_code == 200:
            data = response.json()
            if data.get('features'):
                # Koordinatları al (Photon [boylam, enlem] döndürür)
                coords = data['features'][0]['geometry']['coordinates']
                enlem = coords[1]
                boylam = coords[0]
                return enlem, boylam
        
        return None, None
    except:
        return None, None


def get_coordinates(adres, kutuphane_adi, ilce_adi):
    """
    Farklı adres formatlarını deneyerek koordinat buluyoruz
    """
    # Farklı adres formatları oluştur
    adres_varyantlari = [
        adres,  # Orijinal adres
        f"{kutuphane_adi}, {adres}",  # Kütüphane adı + adres
        f"{adres}, Istanbul, Turkey",  # Adres + şehir + ülke
        f"{kutuphane_adi}, {ilce_adi}, Istanbul",  # Kütüphane + ilçe + şehir
        f"{ilce_adi}, Istanbul, Turkey",  # İlçe + şehir + ülke
    ]
    
    # Her formatı sırayla dene
    for adres_formati in adres_varyantlari:
        enlem, boylam = get_coordinates_from_photon(adres_formati)
        
        # Koordinat bulunduysa döndür
        if enlem and boylam:
            return enlem, boylam
        time.sleep(0.3)
    
    # Hiçbiri bulunamadıysa None döndür
    return None, None

## Adım 4: Koordinat Çekme

In [73]:
# Yeni sütunlar oluştur
df['Enlem'] = None
df['Boylam'] = None

print("Geocoding başlıyor...\n")

# Her kütüphane için koordinat çek
for index, row in df.iterrows():
    adres = row['Adres']
    kutuphane_adi = row['Kütüphane Adı']
    ilce_adi = row['İlçe Adı']
    
    # Koordinatları al (farklı formatları deneyerek)
    enlem, boylam = get_coordinates(adres, kutuphane_adi, ilce_adi)
    
    # DataFrame'e kaydet
    df.at[index, 'Enlem'] = enlem
    df.at[index, 'Boylam'] = boylam
    
    # Ekrana yazdır
    print(f"{index+1}. {kutuphane_adi}: Enlem={enlem}, Boylam={boylam}")

# Özet bilgi
print(f"\nToplam: {len(df)} kütüphane")
print(f"Başarılı: {df['Enlem'].notna().sum()} kütüphane")
print(f"Başarısız: {df['Enlem'].isna().sum()} kütüphane")

Geocoding başlıyor...

1. Atatürk Kitaplığı (Merkez Kütüphane): Enlem=41.0391363, Boylam=28.989522071612836
2. Kadın Eserleri Kütüphanesi: Enlem=41.0085416, Boylam=28.9716244
3. Barış Manço Kütüphanesi: Enlem=41.05454465, Boylam=28.872532399999997
4. Eyüp Sultan Kütüphanesi: Enlem=41.0467924, Boylam=28.9360652
5. Ahmet Süheyl Ünver Kütüphanesi: Enlem=41.0249223, Boylam=29.1046151
6. Osman Nuri Ergin Kütüphanesi: Enlem=41.0285491, Boylam=28.8770823
7. İdris Güllüce Kütüphanesi: Enlem=40.850046750000004, Boylam=29.30311591197897
8. Muallim Cevdet Kütüphanesi: Enlem=40.88832925, Boylam=29.190645475602643
9. Halil İnalcık Kütüphanesi: Enlem=40.905902, Boylam=29.1613794
10. Sesli Kütüphane: Enlem=41.0482431, Boylam=28.9354595
11. Erdem Bayazıt Kütüphanesi: Enlem=41.178701, Boylam=28.702508
12. Metin And Kütüphanesi: Enlem=40.997982449999995, Boylam=28.792749537310314
13. Ahmet Kabaklı Kütüphanesi: Enlem=41.01962815, Boylam=28.939689451697102
14. Rasim Özdenören Kütüphanesi: Enlem=41.0753986

## Adım 5: Gerekli Sütunları Seçme

In [75]:
# Sadece gerekli sütunları seç
df_filtered = df[["Kütüphane Adı", "Enlem", "Boylam"]].copy()

print(f"Filtrelenmiş veri seti: {len(df_filtered)} kütüphane")
df_filtered.head()

Filtrelenmiş veri seti: 60 kütüphane


,Kütüphane Adı,Enlem,Boylam
0,Atatürk Kitaplığı (Merkez Kütüphane),41.039136,28.989522
1,Kadın Eserleri Kütüphanesi,41.008542,28.971624
2,Barış Manço Kütüphanesi,41.054545,28.872532
3,Eyüp Sultan Kütüphanesi,41.046792,28.936065
4,Ahmet Süheyl Ünver Kütüphanesi,41.024922,29.104615


## Adım 6: JSON Olarak Kaydetme

In [ ]:
# JSON dosyasına kaydet
output_file = "adresler.json"
df_filtered.to_json(output_file, orient='records', force_ascii=False, indent=4)

print(f" {len(df_filtered)} kütüphane '{output_file}' dosyasına kaydedildi.")

 60 kütüphane 'adresler.json' dosyasına kaydedildi.
